In [2]:
import pandas as pd
file = "covspectrum.csv"
df = pd.read_csv(file)

file2 = "biosamples.txt"
cols = ["genbankAccession", "biosample"]
df2 = pd.read_csv(file2, sep="\t", names=cols)

df = df.merge(df2, on="genbankAccession")

In [3]:
# filter to those with biosample
df = df[df.biosample.notnull()]


In [4]:
# pick 500 random biosamples
sampled = df.sample(100)
biosamples = sampled.biosample.values

In [5]:
import tqdm
import subprocess
import json
import os

os.system("rm *.fastq.gz")
os.system("rm fastq/*.fastq.gz")

for biosample in tqdm.tqdm(biosamples):
    print(biosample)
    # use subprocess like:  ffq SRR18665696 --ftp to get JSON urls
    try:
        subprocess.check_output(["./enaFastqFetch.py","-s", biosample, "-d","sample"])
    except Exception:
        print("e")

os.system("mv *.fastq.gz fastq")

  0%|          | 0/100 [00:00<?, ?it/s]

SAMEA14372866


  1%|          | 1/100 [00:09<14:57,  9.07s/it]

SAMEA13602972


  2%|▏         | 2/100 [00:21<17:43, 10.86s/it]

SAMN25813889


  3%|▎         | 3/100 [00:47<29:00, 17.94s/it]

SAMN25264181


  3%|▎         | 3/100 [00:48<25:53, 16.01s/it]


CalledProcessError: Command '['./enaFastqFetch.py', '-s', 'SAMN25264181', '-d', 'sample']' returned non-zero exit status 1.

In [47]:
# get list of all fastq.gz files
fastq_files = [x for x in os.listdir("fastq") if x.endswith(".gz")]

common_adapters = ["AGATCGGAAGAGCACACGTCTGAACTCCAGTCA",
    "AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT",
    "CTGTCTCTTATACACATCT", "AGATGTGTATAAGAGACAG"]

# trim adapters
out_dir = "trimmed"
for fastq in tqdm.tqdm(fastq_files):
    command = f"cutadapt -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -a AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -a AGATGTGTATAAGAGACAG -m 20 -o {out_dir}/{fastq} fastq/{fastq} -j 20"
    subprocess.call(command, shell=True)

100%|██████████| 42/42 [02:02<00:00,  2.92s/it]


In [48]:

# arrange into pairs where paired
from collections import defaultdict
fastq_pairs = defaultdict(list)
fastq_singles = []
for fastq_file in fastq_files:
    # if contains underscore
    if "_" in fastq_file:
        # split on underscore
        root, number = fastq_file.split("_")
        # add to paired list
        fastq_pairs[root].append(fastq_file)
    else:
        fastq_singles.append(fastq_file)


In [49]:
import sys
import os
# map with minimap2, and make bam with bamtools
for root, files in fastq_pairs.items():
    # trim off adapters

    reference = "ref.fa"
    target = os.path.join("bam", root+".bam")
    # 10 threads
    #pipe to bamtools to make bam
    command = f"minimap2 -t 10 -ax sr {reference} trimmed/{files[0]} trimmed/{files[1]} | samtools view -bS - > {target}"
    print(command)
    os.system(command)
    


minimap2 -t 10 -ax sr ref.fa trimmed/ERR3224171_1.fastq.gz trimmed/ERR3224171_2.fastq.gz | samtools view -bS - > bam/ERR3224171.bam
minimap2 -t 10 -ax sr ref.fa trimmed/ERR3601708_1.fastq.gz trimmed/ERR3601708_2.fastq.gz | samtools view -bS - > bam/ERR3601708.bam
minimap2 -t 10 -ax sr ref.fa trimmed/ERR4324960_1.fastq.gz trimmed/ERR4324960_2.fastq.gz | samtools view -bS - > bam/ERR4324960.bam
minimap2 -t 10 -ax sr ref.fa trimmed/ERR4325088_1.fastq.gz trimmed/ERR4325088_2.fastq.gz | samtools view -bS - > bam/ERR4325088.bam
minimap2 -t 10 -ax sr ref.fa trimmed/SRR10190436_1.fastq.gz trimmed/SRR10190436_2.fastq.gz | samtools view -bS - > bam/SRR10190436.bam
minimap2 -t 10 -ax sr ref.fa trimmed/SRR10726961_1.fastq.gz trimmed/SRR10726961_2.fastq.gz | samtools view -bS - > bam/SRR10726961.bam
minimap2 -t 10 -ax sr ref.fa trimmed/SRR10740518_1.fastq.gz trimmed/SRR10740518_2.fastq.gz | samtools view -bS - > bam/SRR10740518.bam
minimap2 -t 10 -ax sr ref.fa trimmed/SRR10788409_1.fastq.gz trimmed

In [51]:
# now do single end
for fastq_file in fastq_singles:
    reference = "ref.fa"
    target = os.path.join("bam", fastq_file.replace(".gz", ".bam"))
    command = f"minimap2 -t 10 -ax sr {reference} trimmed/{fastq_file} | samtools view -bS - > {target}"
    print(command)
    os.system(command)

minimap2 -t 10 -ax sr ref.fa trimmed/DRR093148.fastq.gz | samtools view -bS - > bam/DRR093148.fastq.bam
minimap2 -t 10 -ax sr ref.fa trimmed/ERR2031146.fastq.gz | samtools view -bS - > bam/ERR2031146.fastq.bam
minimap2 -t 10 -ax sr ref.fa trimmed/ERR3793839.fastq.gz | samtools view -bS - > bam/ERR3793839.fastq.bam
minimap2 -t 10 -ax sr ref.fa trimmed/SRR10737212.fastq.gz | samtools view -bS - > bam/SRR10737212.fastq.bam
minimap2 -t 10 -ax sr ref.fa trimmed/SRR10868344.fastq.gz | samtools view -bS - > bam/SRR10868344.fastq.bam
minimap2 -t 10 -ax sr ref.fa trimmed/SRR11302877.fastq.gz | samtools view -bS - > bam/SRR11302877.fastq.bam
minimap2 -t 10 -ax sr ref.fa trimmed/SRR11302903.fastq.gz | samtools view -bS - > bam/SRR11302903.fastq.bam
minimap2 -t 10 -ax sr ref.fa trimmed/SRR9731045.fastq.gz | samtools view -bS - > bam/SRR9731045.fastq.bam


In [52]:
# for each bam file, sort and index
for bam_file in os.listdir("bam"):
    if bam_file.endswith(".bam"):
        command = f"samtools sort bam/{bam_file} > bam/{bam_file.replace('.bam', '.sorted.bam')}"
        print(command)
        os.system(command)
        command = f"samtools index bam/{bam_file.replace('.bam', '.sorted.bam')}"
        print(command)
        os.system(command)

samtools sort bam/DRR093148.fastq.bam > bam/DRR093148.fastq.sorted.bam
samtools index bam/DRR093148.fastq.sorted.bam
samtools sort bam/DRR093148.fastq.sorted.bam > bam/DRR093148.fastq.sorted.sorted.bam
samtools index bam/DRR093148.fastq.sorted.sorted.bam
samtools sort bam/ERR2031146.fastq.bam > bam/ERR2031146.fastq.sorted.bam
samtools index bam/ERR2031146.fastq.sorted.bam
samtools sort bam/ERR2031146.fastq.sorted.bam > bam/ERR2031146.fastq.sorted.sorted.bam
samtools index bam/ERR2031146.fastq.sorted.sorted.bam
samtools sort bam/ERR3224171.bam > bam/ERR3224171.sorted.bam
samtools index bam/ERR3224171.sorted.bam
samtools sort bam/ERR3224171.sorted.bam > bam/ERR3224171.sorted.sorted.bam
samtools index bam/ERR3224171.sorted.sorted.bam
samtools sort bam/ERR3601708.bam > bam/ERR3601708.sorted.bam
samtools index bam/ERR3601708.sorted.bam
samtools sort bam/ERR3601708.sorted.bam > bam/ERR3601708.sorted.sorted.bam
samtools index bam/ERR3601708.sorted.sorted.bam
samtools sort bam/ERR3793839.fastq

In [44]:
# get coverage per base for each bam file
for bam_file in os.listdir("bam"):
    if bam_file.endswith(".sorted.bam"):
        start = 20000
        end = 25000
        command = f"samtools depth -r {start}-{end} bam/{bam_file} > coverage/{bam_file.replace('.sorted.bam', '.coverage')}"
        print(command)
        os.system(command)
        

samtools depth -r 20000-25000 bam/DRR093148.fastq.sorted.bam > coverage/DRR093148.fastq.coverage
samtools depth -r 20000-25000 bam/ERR2031146.fastq.sorted.bam > coverage/ERR2031146.fastq.coverage
samtools depth -r 20000-25000 bam/ERR3224171.sorted.bam > coverage/ERR3224171.coverage
samtools depth -r 20000-25000 bam/ERR3601708.sorted.bam > coverage/ERR3601708.coverage
samtools depth -r 20000-25000 bam/ERR3793839.fastq.sorted.bam > coverage/ERR3793839.fastq.coverage
samtools depth -r 20000-25000 bam/ERR4324960.sorted.bam > coverage/ERR4324960.coverage
samtools depth -r 20000-25000 bam/ERR4325088.sorted.bam > coverage/ERR4325088.coverage
samtools depth -r 20000-25000 bam/SRR10190436.sorted.bam > coverage/SRR10190436.coverage
samtools depth -r 20000-25000 bam/SRR10726961.sorted.bam > coverage/SRR10726961.coverage
samtools depth -r 20000-25000 bam/SRR10737212.fastq.sorted.bam > coverage/SRR10737212.fastq.coverage
samtools depth -r 20000-25000 bam/SRR10740518.sorted.bam > coverage/SRR1074051